# Trade History analysis
## Objectives:
### Data Processing
#### Split data:
- Index and Non-index split
- For Index, strategies: Covered Index, Index Strangle.
- For all other: Strangle, portfolio call short.(NaN = "strangle")
#### Cleanup:
- Handle Null/NaN
- Handle Data Type
- Drop Columns
- Drop Unwanted Strategy
- Encode columns: ['For ', 'Closed ', 'Transaction Type ', 'Call Type ' ]
- Rename '% Diff Cur Price N Strike Price' to 'Diff_TDP/SP'

#### 1. Analysing trade behaviour
- Price-to-cover instances.(How many times have trades been made within the price to cover price.)
- Price-to-cover deviation.(How much deviation is there between executed trades and respective price-to=cover points.)
#### 2. Profitability analysis
- Price-to-cover deviation and profitability
- Loss indicator identification
    - Is there a relationship between the incidence of loss and the stock ATR?
    - What is the strength of this relationship
    - Are there any other indicators that can predict the trade outcome better than ATR
#### 3. Pattern Identification
- What have been the instances of major loss?
- Is there a pattern that describes the probability of loss
- What is the current Expected to realized investment outcome relationship?
- What is the realistic model that can help describe monthly investment outcome.

In [1002]:
#package import
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math as math
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [1003]:
#Reading data:
data_uncleaned = pd.read_csv('options_data_25.csv')
data_uncleaned.head()

,Trade Closed,For,Expiry Date,Date,Strangle/ Cover,Strategy Name,Instrument,LotSize ChangeMonth,Call Type,Transaction Type,...,Qty,Sold Rate,Bought,Bought Date,Total Premium,%,Sold Value,Bought Value,Profit,% Diff Cur Price N Strike Price
0,Closed,Mohan,30/Apr/20,1/Apr/20,NaN,NaN,HDFCBANK,NaN,CALL,SELL,...,550,4.35,0.45,24/Apr/20,2145.00,0.0%,2392.50,247.50,2145,30.9%
1,Closed,Mohan,30/Apr/20,1/Apr/20,NaN,NaN,NIFTY,NaN,CALL,BUY,...,75,6.95,12.45,1/Apr/20,412.50,0.0%,521.25,933.75,-412.5,31.2%
2,Closed,Mohan,30/Apr/20,1/Apr/20,NaN,NaN,NIFTY,NaN,CALL,BUY,...,75,7.00,12.45,1/Apr/20,408.75,0.0%,525.00,933.75,-408.75,31.2%
3,Closed,Mohan,28/May/20,14/May/20,NaN,NaN,ASIANPAINT,NaN,PUT,SELL,...,300,13.75,4.00,NaN,2925.00,4.2%,4125.00,1200.00,2925,-8.8%
4,Closed,Mohan,28/May/20,15/May/20,NaN,NaN,ASIANPAINT,NaN,CALL,SELL,...,300,3.45,1.45,NaN,600.00,1.7%,1035.00,435.00,600,15.7%


In [1004]:
data_uncleaned.columns

Index(['Trade Closed', 'For', 'Expiry Date', 'Date', 'Strangle/ Cover',
       'Strategy Name', 'Instrument', 'LotSize ChangeMonth', 'Call Type',
       'Transaction Type', 'Lot', 'Strike Price', 'Current Prices',
       '% Gap between Strike Price and Current Price', 'Option Status',
       'Change since Trade Taken', 'Support /Resistence with Renko', 'Price',
       'Bollinger Band (High)', 'Bollinger Band (Low)', 'ATR',
       'Atr % of Price', 'Higher Level Price based on ATR',
       'Possible Price High', 'Lower Level Price based on ATR',
       'Possible Price Low', 'Price to Cover', 'Transaction status',
       'Margin Required', 'Qty', 'Sold Rate', 'Bought', 'Bought Date',
       'Total Premium', '%', 'Sold Value', 'Bought Value', ' Profit ',
       '% Diff Cur Price N Strike Price'],
      dtype='object')

In [1005]:
data_uncleaned.rename(columns={'% Diff Cur Price N Strike Price': 'Diff_TDP/SP'},  inplace=True)

In [1006]:
data_uncleaned.dtypes

Trade Closed                                     object
For                                              object
Expiry Date                                      object
Date                                             object
Strangle/ Cover                                 float64
Strategy Name                                    object
Instrument                                       object
LotSize ChangeMonth                              object
Call Type                                        object
Transaction Type                                 object
Lot                                               int64
Strike Price                                    float64
Current Prices                                  float64
% Gap between Strike Price and Current Price     object
Option Status                                    object
Change since Trade Taken                         object
Support /Resistence with Renko                   object
Price                                           

In [1007]:
#fixing datatypes
data_uncleaned["Expiry Date"] = pd.to_datetime(data_uncleaned["Expiry Date"], format='mixed')

In [1008]:
data_uncleaned["Expiry Date"]

0      2020-04-30
1      2020-04-30
2      2020-04-30
3      2020-05-28
4      2020-05-28
          ...    
6345   2025-04-24
6346   2025-04-24
6347   2025-04-24
6348   2025-04-24
6349   2025-04-24
Name: Expiry Date, Length: 6350, dtype: datetime64[ns]

In [1009]:
data_uncleaned['Date']= pd.to_datetime(data_uncleaned["Date"], format = 'mixed', errors='coerce')

In [1010]:
data_uncleaned["Date"]

0      2020-04-01
1      2020-04-01
2      2020-04-01
3      2020-05-14
4      2020-05-15
          ...    
6345   2025-03-28
6346   2025-03-28
6347   2025-03-28
6348   2025-03-28
6349   2025-03-28
Name: Date, Length: 6350, dtype: datetime64[ns]

In [1011]:
data_uncleaned.dtypes

Trade Closed                                            object
For                                                     object
Expiry Date                                     datetime64[ns]
Date                                            datetime64[ns]
Strangle/ Cover                                        float64
Strategy Name                                           object
Instrument                                              object
LotSize ChangeMonth                                     object
Call Type                                               object
Transaction Type                                        object
Lot                                                      int64
Strike Price                                           float64
Current Prices                                         float64
% Gap between Strike Price and Current Price            object
Option Status                                           object
Change since Trade Taken                               

In [1012]:
data_uncleaned["For"] = np.where(data_uncleaned["For"].str.contains('Mohan',case=False),0,1)

In [1013]:
data_uncleaned["For"].head()

0    0
1    0
2    0
3    0
4    0
Name: For, dtype: int64

In [1014]:
data_uncleaned["Trade Closed"] = np.where(data_uncleaned["Trade Closed"].str.contains('Closed',case=False),0,1)

In [1015]:
data_uncleaned["Trade Closed"].tail()

6345    1
6346    1
6347    1
6348    1
6349    1
Name: Trade Closed, dtype: int64

In [1016]:
print(data_uncleaned["Strangle/ Cover"].unique())
print(data_uncleaned["Strategy Name"].unique())

[nan  1.  2.  3.  4.]
[nan 'Strangle' 'DeltaNeutral' 'Naked Directional' 'Investment'
 'Directional' 'Protection' 'Stradle' 'Portfolio' 'BuyBack' 'WHEEL'
 'Borrowed C' 'Index Strangle' 'Portfolio Call Short' 'Covered PUT'
 'Covered Strangle' 'Covered CALL' 'Covered Put Bearish' 'Covered Stradle'
 'BANKNIFTY' 'RICHIE' 'Ban Strategy' 'Covered Put'
 'DoubleTop Covered Strangle' 'HeroZero' 'Result Trade' 'Pair Trade'
 'Covered Strangle Yearly' 'Budget' 'Iron Fly' 'Synthetic Future'
 'Directional Rajesh' 'Future Spread' 'Always In Trade' 'Covered Trade'
 'Combo' 'Covered Index' 'Directional Zarir' 'Directional MACD Strategy'
 'Directional Sangam' 'Expiry' 'NIFTY STOCK Trade' 'Index Arbitrage'
 'BANKNIFTY Stock Trade' 'Ladder Put' 'Index Hedge' 'Bearish Directional'
 'Ladder Call' 'Ladder Strangle' 'Easy Options Expiry' 'Iron Butterfly'
 'Iron Condor' 'Weekly Index' 'Synthetic Short' 'Index Covered Yearly']


In [1017]:
data_uncleaned["Strangle/ Cover"] = data_uncleaned["Strangle/ Cover"].fillna(0).astype(int)

In [1018]:
data_uncleaned["Strangle/ Cover"] = data_uncleaned["Strangle/ Cover"].astype(int)

In [1019]:
data_uncleaned['Transaction Type'] = np.where(data_uncleaned['Transaction Type'].str.contains('Sell',case = False),0,1)

In [1020]:
data_uncleaned['Call Type'] = np.where(data_uncleaned['Call Type'].str.contains('Call',case = False),0,1)

In [1021]:
data_uncleaned['Call Type'].unique()

array([0, 1])

In [1022]:
# list of columns to astype(int): Margin required bollinger band high

In [1023]:
Na_count = pd.DataFrame(data_uncleaned.isnull().sum())

In [1024]:
data_uncleaned['Trade Closed'].value_counts()

Trade Closed
0    6309
1      41
Name: count, dtype: int64

In [1025]:
data_uncleaned.shape

(6350, 39)

In [1026]:
closed = data_uncleaned[data_uncleaned['Trade Closed']==0].copy()

In [1027]:
closed.head()
closed.shape

(6309, 39)

In [1028]:
col_to_drop = ['LOT Quantity','LotSize ChangeMonth','Option Status','Change since Trade Taken','Support /Resistence with Renko','Higher Level Price based on ATR','Possible Price High','Lower Level Price based on ATR','Possible Price Low','Transaction status','Margin Required','Qty','Sold Rate','Bought','Bought Date','Total Premium','ChangedLotSize','Num_Chars','Call type Tag','Instrument Group2','Periodicity','First Bought Date','Starting Price','Vol % of Total Shares','Change (%)','StrangleCoverTotal Premium','StrangleCover Margin','ExpirtyDateTrade','Remarks','Column1','Ratio','Column2','Known Max Loss','10% range','Max possible Loss','% Gap between Strike Price and Current Price','Transaction Type','Lot','Active','For','StrangleCover Profit','Script','%',' Ratio ', 'Stock Name']

In [1029]:
closed.columns

Index(['Trade Closed', 'For', 'Expiry Date', 'Date', 'Strangle/ Cover',
       'Strategy Name', 'Instrument', 'LotSize ChangeMonth', 'Call Type',
       'Transaction Type', 'Lot', 'Strike Price', 'Current Prices',
       '% Gap between Strike Price and Current Price', 'Option Status',
       'Change since Trade Taken', 'Support /Resistence with Renko', 'Price',
       'Bollinger Band (High)', 'Bollinger Band (Low)', 'ATR',
       'Atr % of Price', 'Higher Level Price based on ATR',
       'Possible Price High', 'Lower Level Price based on ATR',
       'Possible Price Low', 'Price to Cover', 'Transaction status',
       'Margin Required', 'Qty', 'Sold Rate', 'Bought', 'Bought Date',
       'Total Premium', '%', 'Sold Value', 'Bought Value', ' Profit ',
       'Diff_TDP/SP'],
      dtype='object')

In [1030]:
def check_col(df,col_names):
    exist = []
    for i in range(len(closed.columns)):
        if df.columns[i] in col_to_drop:
            exist.append(closed.columns[i])
        #else:
            #print(f'Keeping column {closed.columns[i]}')
    return exist

print(check_col(closed,col_to_drop))

['For', 'LotSize ChangeMonth', 'Transaction Type', 'Lot', '% Gap between Strike Price and Current Price', 'Option Status', 'Change since Trade Taken', 'Support /Resistence with Renko', 'Higher Level Price based on ATR', 'Possible Price High', 'Lower Level Price based on ATR', 'Possible Price Low', 'Transaction status', 'Margin Required', 'Qty', 'Sold Rate', 'Bought', 'Bought Date', 'Total Premium', '%']


In [1031]:
cleaned = pd.DataFrame(closed.drop(check_col(closed,col_to_drop),axis=1))

In [1032]:
cleaned.columns

Index(['Trade Closed', 'Expiry Date', 'Date', 'Strangle/ Cover',
       'Strategy Name', 'Instrument', 'Call Type', 'Strike Price',
       'Current Prices', 'Price', 'Bollinger Band (High)',
       'Bollinger Band (Low)', 'ATR', 'Atr % of Price', 'Price to Cover',
       'Sold Value', 'Bought Value', ' Profit ', 'Diff_TDP/SP'],
      dtype='object')

In [1033]:
cleaned.head()

,Trade Closed,Expiry Date,Date,Strangle/ Cover,Strategy Name,Instrument,Call Type,Strike Price,Current Prices,Price,Bollinger Band (High),Bollinger Band (Low),ATR,Atr % of Price,Price to Cover,Sold Value,Bought Value,Profit,Diff_TDP/SP
0,0,2020-04-30,2020-04-01,0,NaN,HDFCBANK,0,1200.0,1828.20,830,NaN,NaN,NaN,0%,0,2392.50,247.50,2145,30.9%
1,0,2020-04-30,2020-04-01,0,NaN,NIFTY,0,12000.0,23519.35,8254,NaN,NaN,NaN,0%,0,521.25,933.75,-412.5,31.2%
2,0,2020-04-30,2020-04-01,0,NaN,NIFTY,0,12000.0,23519.35,8254,NaN,NaN,NaN,0%,0,525.00,933.75,-408.75,31.2%
3,0,2020-05-28,2020-05-14,0,NaN,ASIANPAINT,1,1400.0,2340.65,1523,2105,NaN,73.0,5%,1443,4125.00,1200.00,2925,-8.8%
4,0,2020-05-28,2020-05-15,0,NaN,ASIANPAINT,0,1800.0,2340.65,1518,2105,NaN,73.0,5%,2105,1035.00,435.00,600,15.7%


In [1034]:
cleaned.dtypes

Trade Closed                      int64
Expiry Date              datetime64[ns]
Date                     datetime64[ns]
Strangle/ Cover                   int64
Strategy Name                    object
Instrument                       object
Call Type                         int64
Strike Price                    float64
Current Prices                  float64
Price                            object
Bollinger Band (High)            object
Bollinger Band (Low)            float64
ATR                             float64
Atr % of Price                   object
Price to Cover                    int64
Sold Value                      float64
Bought Value                    float64
 Profit                          object
Diff_TDP/SP                      object
dtype: object

In [1035]:
cleaned[" Profit "]=cleaned[' Profit '].str.replace(",","")
cleaned[' Profit ']=cleaned[' Profit '].str.replace("(","-")
cleaned[' Profit ']=cleaned[' Profit '].str.replace(")","")


In [1036]:
cleaned[' Profit ']=cleaned[' Profit '].astype(float)

In [1037]:
cleaned['Atr % of Price']=cleaned["Atr % of Price"].str.replace('%','').astype(float)

In [1038]:
cleaned['Bollinger Band (High)'] = cleaned["Bollinger Band (High)"].fillna(0)


In [1039]:
cleaned["Bollinger Band (High)"] = cleaned["Bollinger Band (High)"].replace(["", " "], np.nan)
cleaned['Bollinger Band (High)'] = cleaned["Bollinger Band (High)"].str.replace("","").astype(float)

In [1040]:
cleaned['Strategy Name'].unique()


array([nan, 'Strangle', 'DeltaNeutral', 'Naked Directional', 'Investment',
       'Directional', 'Protection', 'Stradle', 'Portfolio', 'BuyBack',
       'WHEEL', 'Borrowed C', 'Index Strangle', 'Portfolio Call Short',
       'Covered PUT', 'Covered Strangle', 'Covered CALL',
       'Covered Put Bearish', 'Covered Stradle', 'BANKNIFTY', 'RICHIE',
       'Ban Strategy', 'Covered Put', 'DoubleTop Covered Strangle',
       'HeroZero', 'Result Trade', 'Pair Trade',
       'Covered Strangle Yearly', 'Budget', 'Iron Fly',
       'Synthetic Future', 'Directional Rajesh', 'Future Spread',
       'Always In Trade', 'Covered Trade', 'Combo', 'Covered Index',
       'Directional Zarir', 'Directional MACD Strategy',
       'Directional Sangam', 'Expiry', 'NIFTY STOCK Trade',
       'Index Arbitrage', 'BANKNIFTY Stock Trade', 'Ladder Put',
       'Index Hedge', 'Bearish Directional', 'Ladder Call',
       'Ladder Strangle', 'Easy Options Expiry', 'Iron Butterfly',
       'Iron Condor', 'Weekly Index

In [1041]:
cleaned['Strategy Name']= np.where(cleaned['Strategy Name'].isna(),'Strangle',cleaned['Strategy Name'])

In [1042]:
cleaned.isna().sum()

Trade Closed                0
Expiry Date                 0
Date                        0
Strangle/ Cover             0
Strategy Name               0
Instrument                  0
Call Type                   0
Strike Price                8
Current Prices              0
Price                       0
Bollinger Band (High)    4829
Bollinger Band (Low)     4835
ATR                      4824
Atr % of Price              0
Price to Cover              0
Sold Value                  0
Bought Value                0
 Profit                     0
Diff_TDP/SP                 0
dtype: int64

In [1043]:
#cleaned = cleaned.drop(string_col.columns,axis=1)

In [1044]:
cleaned.shape

(6309, 19)

In [1045]:
cleaned.dtypes

Trade Closed                      int64
Expiry Date              datetime64[ns]
Date                     datetime64[ns]
Strangle/ Cover                   int64
Strategy Name                    object
Instrument                       object
Call Type                         int64
Strike Price                    float64
Current Prices                  float64
Price                            object
Bollinger Band (High)           float64
Bollinger Band (Low)            float64
ATR                             float64
Atr % of Price                  float64
Price to Cover                    int64
Sold Value                      float64
Bought Value                    float64
 Profit                         float64
Diff_TDP/SP                      object
dtype: object

In [1046]:
cleaned['Price'].str.contains('#VALUE!').count()

np.int64(6309)

In [1047]:
def find_missing(df):
    data = df.select_dtypes(include=['object'])
    num_count = {}
    for key in data.columns:
        print(key)
        num_count[key] = data[key].astype(str).str.contains("#VALUE!", na=False).sum()
    return num_count


In [1048]:
find_missing(cleaned)

Strategy Name
Instrument
Price
Diff_TDP/SP


{'Strategy Name': np.int64(0),
 'Instrument': np.int64(0),
 'Price': np.int64(34),
 'Diff_TDP/SP': np.int64(34)}

In [1049]:
cleaned["Price"].replace({'#VALUE!': np.nan}, inplace=True)

In [1050]:
cleaned["Price"].astype(str).str.replace('%', '', regex=False)

0         830
1        8254
2        8254
3        1523
4        1518
        ...  
6308    23813
6309    23813
6310    23813
6311    23813
6312    23668
Name: Price, Length: 6309, dtype: object

In [1051]:
cleaned["Price"].astype(str).str.replace('86765.00%', '86765.00', regex=False)

0         830
1        8254
2        8254
3        1523
4        1518
        ...  
6308    23813
6309    23813
6310    23813
6311    23813
6312    23668
Name: Price, Length: 6309, dtype: object

In [1052]:
cleaned['Diff_TDP/SP'] = cleaned['Diff_TDP/SP'].replace({'#DIV/0!': np.nan})

In [1053]:
find_missing(cleaned)

Strategy Name
Instrument
Price
Diff_TDP/SP


{'Strategy Name': np.int64(0),
 'Instrument': np.int64(0),
 'Price': np.int64(0),
 'Diff_TDP/SP': np.int64(34)}

In [1054]:
cleaned["Diff_TDP/SP"] = cleaned['Diff_TDP/SP'].astype(str).str.replace('%',"",regex=False)

In [1055]:
cleaned['Diff_TDP/SP'].head(100)

0      30.9
1      31.2
2      31.2
3      -8.8
4      15.7
      ...  
95     -1.5
96    -10.0
97     -8.8
98    -10.6
99     -7.4
Name: Diff_TDP/SP, Length: 100, dtype: object

In [1056]:
cleaned["Diff_TDP/SP"]=cleaned["Diff_TDP/SP"].replace({'#VALUE!': np.nan})

In [1057]:
cleaned['Diff_TDP/SP'] = cleaned['Diff_TDP/SP'].astype(float)

In [1058]:
cleaned['Price'] = cleaned['Price'].astype(str).str.replace('%', '', regex=False)

In [1059]:
cleaned.shape

(6309, 19)

In [1060]:
cleaned.dtypes

Trade Closed                      int64
Expiry Date              datetime64[ns]
Date                     datetime64[ns]
Strangle/ Cover                   int64
Strategy Name                    object
Instrument                       object
Call Type                         int64
Strike Price                    float64
Current Prices                  float64
Price                            object
Bollinger Band (High)           float64
Bollinger Band (Low)            float64
ATR                             float64
Atr % of Price                  float64
Price to Cover                    int64
Sold Value                      float64
Bought Value                    float64
 Profit                         float64
Diff_TDP/SP                     float64
dtype: object

In [1061]:
cleaned['Price']=cleaned['Price'].astype(float)

In [1062]:
cleaned.columns

Index(['Trade Closed', 'Expiry Date', 'Date', 'Strangle/ Cover',
       'Strategy Name', 'Instrument', 'Call Type', 'Strike Price',
       'Current Prices', 'Price', 'Bollinger Band (High)',
       'Bollinger Band (Low)', 'ATR', 'Atr % of Price', 'Price to Cover',
       'Sold Value', 'Bought Value', ' Profit ', 'Diff_TDP/SP'],
      dtype='object')

In [1063]:
cleaned.dtypes

Trade Closed                      int64
Expiry Date              datetime64[ns]
Date                     datetime64[ns]
Strangle/ Cover                   int64
Strategy Name                    object
Instrument                       object
Call Type                         int64
Strike Price                    float64
Current Prices                  float64
Price                           float64
Bollinger Band (High)           float64
Bollinger Band (Low)            float64
ATR                             float64
Atr % of Price                  float64
Price to Cover                    int64
Sold Value                      float64
Bought Value                    float64
 Profit                         float64
Diff_TDP/SP                     float64
dtype: object

In [1089]:
index_data = cleaned[cleaned['Instrument'].str.contains('Nifty', case = False)].copy()
index_data.head()
index_data['Instrument'].value_counts()
non_index_data = cleaned[~cleaned['Instrument'].str.contains('Nifty', case=False, na=False)]

index_data.shape

(1224, 19)

In [1090]:
non_index_data['Instrument'].unique()

array(['HDFCBANK', 'ASIANPAINT', 'BHARTIARTL', 'ITC', 'KOTAKBANK',
       'NESTLEIND', 'BAJFINANCE', 'ESCORTS', 'SBIN', 'TATAPOWER', 'TCS',
       'BHEL', 'COALINDIA', 'TATAMOTORS', 'TITAN', 'CIPLA', 'TATACONSUM',
       'CADILAHC', 'DIVISLAB', 'GLENMARK', 'JSWSTEEL', 'TATACHEM',
       'TATASTEEL', 'HDFCLIFE', 'HINDUNILVR', 'PEL', 'PNB', 'SAIL',
       'BAJAJFINSV', 'DEEPAKNTR', 'LTI', 'MARUTI', 'INFY', 'ABBOTINDIA',
       'BATAINDIA', 'DLF', 'HINDALCO', 'MANAPPURAM', 'APOLLOTYRE',
       'BANKBARODA', 'SUNPHARMA', 'DABUR', 'NAUKRI', 'ICICIBANK',
       'NATIONALUM\xa0', 'CHOLAFIN', 'BEL', 'CANBK', 'JUBLFOOD', 'LTTS',
       'PIDILITIND', 'INDHOTEL', 'HAL', 'ASTRAL', 'M&M', 'BHARATFORG',
       'AUROPHARMA', 'VOLTAS', 'LT', 'CUMMINSIND', 'ZYDUSLIFE', 'PAGEIND',
       'NTPC', 'BPCL', 'GAIL', 'POLYCAB', 'POWERGRID', 'PERSISTENT',
       'PFC', 'SIEMENS', 'AXISBANK', 'BAJAJAUTO', 'ABCAPITAL', 'COFORGE',
       'HEROMOTOCO', 'LUPIN', 'PIIND', 'SYNGENE', 'UBL', 'ABFRL',
       'INDUSINDB

In [1091]:
index_data['Price to Cover'].isnull().sum()

np.int64(0)

In [1092]:
index_data['Strategy Name'].unique()

array(['Strangle', 'Index Strangle', 'BANKNIFTY', 'RICHIE', 'Directional',
       'Covered CALL', 'HeroZero', 'Covered PUT', 'Pair Trade',
       'Portfolio Call Short', 'Covered Strangle Yearly',
       'Covered Strangle', 'Budget', 'Iron Fly', 'Directional Rajesh',
       'Future Spread', 'Always In Trade', 'Covered Index', 'Expiry',
       'NIFTY STOCK Trade', 'Index Arbitrage', 'BANKNIFTY Stock Trade',
       'Directional Sangam', 'Index Hedge', 'Ladder Put',
       'Ladder Strangle', 'Easy Options Expiry', 'Weekly Index',
       'Ladder Call', 'Index Covered Yearly'], dtype=object)

In [1093]:
index_data = index_data[index_data['Strategy Name'].str.contains('Covered Index|Index Strangle', case =False, na=False)]

In [1094]:
index_data['Strategy Name'].value_counts()

Strategy Name
Index Strangle    101
Covered Index      87
Name: count, dtype: int64

In [1095]:
index_data['Price to Cover'].isnull().sum()

np.int64(0)

In [1096]:
id_columns = index_data.columns

In [1097]:
index_data['distance'] = index_data.apply(
    lambda row: np.nan if row['Price to Cover'] == 0
    else ((row['Strike Price'] - row['Price to Cover'])*100) / row['Price to Cover']
    if row['Call Type'] == 0
    else ((row['Price to Cover'] - row['Strike Price'])*100) / row['Price to Cover'],
axis=1)


In [1099]:
index_data.isnull().sum()

Trade Closed               0
Expiry Date                0
Date                       0
Strangle/ Cover            0
Strategy Name              0
Instrument                 0
Call Type                  0
Strike Price               0
Current Prices             0
Price                      0
Bollinger Band (High)    172
Bollinger Band (Low)     172
ATR                      172
Atr % of Price             0
Price to Cover             0
Sold Value                 0
Bought Value               0
 Profit                    0
Diff_TDP/SP                0
distance                 164
dtype: int64

In [1101]:
index_data.shape

(188, 20)

In [1102]:
non_index_data['Strategy Name'].value_counts()

Strategy Name
Strangle                      2269
Portfolio Call Short           536
Directional                    453
Covered PUT                    295
DoubleTop Covered Strangle     206
Investment                     181
RICHIE                         141
Protection                     137
Ladder Strangle                114
WHEEL                          110
Covered Strangle               108
Directional Rajesh             102
Ladder Put                      62
Covered CALL                    40
Always In Trade                 34
Ladder Call                     32
Borrowed C                      30
Covered Trade                   26
Index Arbitrage                 25
Iron Condor                     23
Iron Butterfly                  18
BANKNIFTY                       16
Ban Strategy                    15
Covered Stradle                 15
Iron Fly                        12
NIFTY STOCK Trade               11
Directional Zarir                9
DeltaNeutral                     8
Future

In [1106]:
non_index_data = non_index_data[non_index_data['Strategy Name'].str.contains('Strangle|portfolio call short', case =False, na=False)]

In [1109]:
non_index_data['Strategy Name'].value_counts()

Strategy Name
Strangle                2269
Portfolio Call Short     536
Name: count, dtype: int64

In [1108]:
non_index_data = non_index_data[~non_index_data['Strategy Name'].str.contains('DoubleTop Covered Strangle|Ladder Strangle|Covered Strangle|Index Strangle', case =False, na=False)]

In [1110]:
non_index_data['distance'] = non_index_data.apply(
    lambda row: np.nan if row['Price to Cover'] == 0
    else ((row['Strike Price'] - row['Price to Cover'])*100) / row['Price to Cover']
    if row['Call Type'] == 0
    else ((row['Price to Cover'] - row['Strike Price'])*100) / row['Price to Cover'],
axis=1)

In [1111]:
non_index_data

,Trade Closed,Expiry Date,Date,Strangle/ Cover,Strategy Name,Instrument,Call Type,Strike Price,Current Prices,Price,Bollinger Band (High),Bollinger Band (Low),ATR,Atr % of Price,Price to Cover,Sold Value,Bought Value,Profit,Diff_TDP/SP,distance
0,0,2020-04-30,2020-04-01,0,Strangle,HDFCBANK,0,1200.0,1828.20,830.0,NaN,NaN,NaN,0.0,0,2392.5,247.5,2145.0,30.9,NaN
3,0,2020-05-28,2020-05-14,0,Strangle,ASIANPAINT,1,1400.0,2340.65,1523.0,2105.0,NaN,73.0,5.0,1443,4125.0,1200.0,2925.0,-8.8,2.979903
4,0,2020-05-28,2020-05-15,0,Strangle,ASIANPAINT,0,1800.0,2340.65,1518.0,2105.0,NaN,73.0,5.0,2105,1035.0,435.0,600.0,15.7,-14.489311
12,0,2020-06-25,2020-05-26,0,Strangle,ASIANPAINT,1,1400.0,2340.65,1633.0,1928.0,1306.0,66.0,4.0,1306,2100.0,885.0,1215.0,-16.6,-7.197550
13,0,2020-06-25,2020-06-12,0,Strangle,ASIANPAINT,1,1400.0,2340.65,1600.0,1804.0,1430.0,57.0,4.0,1430,1335.0,150.0,1185.0,-14.3,2.097902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6277,0,2025-03-27,2025-03-12,1,Strangle,RELIANCE,1,1200.0,1275.10,1257.0,NaN,NaN,NaN,0.0,0,300.0,2475.0,-2175.0,-4.8,NaN
6278,0,2025-03-27,2025-03-12,1,Strangle,RELIANCE,0,1290.0,1275.10,1257.0,NaN,NaN,NaN,0.0,0,3100.0,2875.0,225.0,2.6,NaN
6279,0,2025-03-27,2025-03-12,1,Strangle,RELIANCE,0,1300.0,1275.10,1257.0,NaN,NaN,NaN,0.0,0,8400.0,8200.0,200.0,3.3,NaN
6280,0,2025-03-27,2025-03-12,1,Strangle,RELIANCE,0,1310.0,1275.10,1257.0,NaN,NaN,NaN,0.0,0,3900.0,4500.0,-600.0,4.0,NaN
